In [2]:
%%writefile hillis.cu

#include<bits/stdc++.h>
using namespace std;

__global__ void prefixScanHillis(int* input, int* output, int offset, int size){

    // calculating the index of the array
    int idx = (gridDim.x * blockDim.x) * blockIdx.y + blockDim.x * blockIdx.x + threadIdx.x;
    
    // checking for the index being out of bounds
    if (idx < size){
        output[idx] = input[idx];
        if (idx >= offset){
            output[idx] += input[idx - offset];
        }
    } 
}

int main(){

    // the size of the array
    int size = 8;
    
    printf("The size of the array is: %d \n", size);

    //Create pointers and allocate arrays
    int *input = (int*) malloc(size * sizeof(int));
    int *output = (int*) malloc(size * sizeof(int));
    
    int *arr, *ans;
    
    cudaMalloc(&arr, size * sizeof(int));
    cudaMalloc(&ans, size * sizeof(int));
    
    //Fill the input array with numbers
    for(int i = 0; i < size; i++){ 
        input[i] = i + 1;
    }
    
    printf("\nThe input array is: \n");
    for(int i = 0; i < size; i++){
        cout<<input[i]<<" \t";
    }
    cout<<endl;
    
    printf("\nNow Copying the memory to device.\n");
    cudaMemcpy(arr, input, sizeof(int) * size, cudaMemcpyHostToDevice);
    cudaMemcpy(ans, output, sizeof(int) * size, cudaMemcpyHostToDevice);
    cudaDeviceSynchronize();
    
    int numOfThreads = 1024;
    int blockSize = 2 * numOfThreads;
    int n;
    if(size % blockSize == 0){
        n = size;
    }
    else{
        n = (1 + size / blockSize) * blockSize;
    }
    int gridSize = n/blockSize;
    
    int offset = 1, limit = 2 * size;
    //int gridSize = 1, blockSize = size;
    int* temp;
    while(offset < limit){

        prefixScanHillis <<<gridSize, numOfThreads>>>(arr, ans, offset, size);
        cudaDeviceSynchronize();

        offset = offset * 2;

        temp = arr;
        arr = ans;
        ans = temp;
    }

    printf("\nCopying the results back.\n");
    cudaMemcpy(output, ans, size * sizeof(int), cudaMemcpyDeviceToHost);
    
    printf("\nThe resultant prefixSum array is: \n");
    for(int i = 0; i < size; i++){
        cout<<output[i]<<" \t";
    }
    cout<<endl<<endl;

    //Free the allocated memory
    cudaFree(arr);
    cudaFree(ans);
    
    free(input);
    free(output);
    
    return 0;
}

Overwriting hillis.cu


In [3]:
%%script bash
nvcc ./hillis.cu -o hillis
./hillis

The size of the array is: 8 

The input array is: 
1 	2 	3 	4 	5 	6 	7 	8 	

Now Copying the memory to device.

Copying the results back.

The resultant prefixSum array is: 
1 	3 	6 	10 	15 	21 	28 	36 	



In [4]:
%%writefile bleloch.cu

#include <bits/stdc++.h>
using namespace std;

__global__ void prefixScanBleloch(int* input, int* output, int* pSum, int *sum, int n, int* up)
{
    int blockID = blockIdx.x * blockDim.x;
    int threadID = threadIdx.x;
    
    int arrayID = blockID + threadID;
    int offset = 1;
    
    sum[2 * threadID]   = input[2 * arrayID];
    sum[2 * threadID + 1] = input[2 * arrayID + 1];
    
    // Upsweep
    for(int i = n / 2; i > 0; i = i / 2){
        __syncthreads();
        if(threadID < i){
            int id1 = offset * (2 * threadID + 1) - 1;
            int id2 = offset * (2 * threadID + 2) - 1;
            sum[id2] += sum[id1];
        }
        offset = offset * 2;
    }
    
    up[2 * arrayID] = sum[2 * threadID];
    up[2 * arrayID + 1] = sum[2 * threadID + 1];
    
    if(threadID == 0) {
        if(pSum != NULL){
            pSum[blockIdx.x] = sum[n-1];
        }
        sum[n-1] = 0;
    }
    
    // Downsweep
    for (int i = 1; i < n; i = i * 2){
        offset = offset / 2;
        __syncthreads();
        if (threadID < i){
            int id1 = offset * (2 * threadID + 1) - 1;
            int id2 = offset * (2 * threadID + 2) - 1;
            int temp = sum[id1];
            sum[id1]  = sum[id2];
            sum[id2] += temp;
        }
    }
    __syncthreads();

    output[2 * arrayID] = sum[2 * threadID];
    output[2 * arrayID + 1] = sum[2 * threadID + 1];
}

int main()
{
    // the size of the array
    int size = 8;
    
    printf("The size of the array is: %d \n", size);
    
    //Create pointers and allocate arrays
    int *input = (int*) malloc(size * sizeof(int));
    int *output = (int*) malloc(size * sizeof(int));
    
    int *up = (int*) malloc(size * sizeof(int));
    
    for(int i = 0; i < size; i++){
        input[i] = i + 1;
    }
    
    printf("\nThe input array is: \n");
    for(int i = 0; i < size; i++){
        cout << input[i] << "\t";
    }
    cout<<endl;
    
    int numOfThreads = 1024;
    int blockSize = 2 * numOfThreads;
    int n;
    if(size % blockSize == 0){
        n = size;
    }
    else{
        n = (1 + size / blockSize) * blockSize;
    }
    int gridSize = n/blockSize;
    
    int *arr, *ans, *sum, *newSum, *upPtr;
    cudaMalloc(&arr, sizeof(int) * size);
    cudaMalloc(&upPtr, sizeof(int) * size);
    cudaMalloc(&ans, sizeof(int) * n);
    cudaMalloc(&sum, sizeof(int) * gridSize);
    cudaMalloc(&newSum, sizeof(int) * gridSize);
    
    printf("\nNow Copying the memory to device.\n");
    cudaMemcpy(arr, input, sizeof(int) * size, cudaMemcpyHostToDevice);
    cudaMemcpy(ans, output, sizeof(int) * n, cudaMemcpyHostToDevice);
    cudaMemcpy(upPtr, up, sizeof(int) * n, cudaMemcpyHostToDevice);
    
    printf("\nNow performing scan using cuda\n");
    // perform scan using cuda
    prefixScanBleloch<<<gridSize, numOfThreads>>>(arr, ans, sum, newSum, blockSize, upPtr);
    cudaDeviceSynchronize();
    
    printf("\nCopying the results back.\n");
    // copy scan result back to output
    cudaMemcpy(output, ans, sizeof(int)*size, cudaMemcpyDeviceToHost);
    
    cudaMemcpy(up, upPtr, sizeof(int)*size, cudaMemcpyDeviceToHost);
    
    
    printf("\nThe result of the upsweep is: \n");
    for(int i = 0; i < size; i++){
        cout<<up[i]<<" \t";
    }
    cout<<endl;
    
    printf("\nThe resultant prefixSum array is: \n");
    for(int i = 0; i < size; i++){
        cout<<output[i]<<" \t";
    }
    cout<<endl<<endl;

    //Free the allocated memory
    cudaFree(arr);
    cudaFree(ans);
    cudaFree(sum);
    cudaFree(newSum);
    
    free(input);
    free(output);
    
    return 0;
}

Writing bleloch.cu


In [5]:
%%script bash
nvcc ./bleloch.cu -o bleloch
./bleloch

The size of the array is: 8 

The input array is: 
1	2	3	4	5	6	7	8	

Now Copying the memory to device.

Now performing scan using cuda

Copying the results back.

The result of the upsweep is: 
1 	3 	3 	10 	5 	11 	7 	36 	

The resultant prefixSum array is: 
0 	1 	3 	6 	10 	15 	21 	28 	

